# Access to Hive Metastore

In [3]:
%%sql
SHOW TABLES

In [ ]:
%%sql
SELECT clientid, count(*) as cnt 
FROM hivesampletable 
WHERE state = 'Washington' AND devicemake = 'Microsoft' AND querydwelltime > 15
GROUP BY clientid

Creating SparkContext as 'sc'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1472684042648_0006,pyspark,idle,Link,Link,✔


Creating HiveContext as 'sqlContext'
SparkContext and HiveContext created. Executing user code ...


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


# Data Frames

In [6]:
# hivesampletabledf is a dataframe
hivesampletabledf = sqlContext.table('hivesampletable')

In [5]:
hivesampletabledf.show()

+--------+---------+------+--------------+----------+-----------+-------------+-------------+--------------+---------+--------------------+
|clientid|querytime|market|deviceplatform|devicemake|devicemodel|        state|      country|querydwelltime|sessionid|sessionpagevieworder|
+--------+---------+------+--------------+----------+-----------+-------------+-------------+--------------+---------+--------------------+
|       8| 18:54:20| en-US|       Android|   Samsung|   SCH-i500|   California|United States|    13.9204007|        0|                   0|
|      23| 19:19:44| en-US|       Android|       HTC| Incredible| Pennsylvania|United States|          null|        0|                   0|
|      23| 19:19:46| en-US|       Android|       HTC| Incredible| Pennsylvania|United States|     1.4757422|        0|                   1|
|      23| 19:19:47| en-US|       Android|       HTC| Incredible| Pennsylvania|United States|      0.245968|        0|                   2|
|      28| 01:37:50|

In [11]:
# Use GroupBy clause with dataframe 
hivesampletabledf.groupBy('deviceplatform').count().select('deviceplatform', 'count').orderBy('count', ascending=False).limit(3).show()

+--------------+-----+
|deviceplatform|count|
+--------------+-----+
|       Android|31591|
|     iPhone OS|22731|
|        RIM OS| 3464|
+--------------+-----+

# Data Frames from files

In [9]:
from pyspark.sql.types import *
import csv
from StringIO import StringIO
def csv_values_in_line(line):
    sio = StringIO(line)
    value = csv.reader(sio).next()
    sio.close()
    return value
buildings = sc.textFile('wasb:///HdiSamples/HdiSamples/SensorSampleData/building/building.csv')\
              .map(csv_values_in_line)\
              .filter(lambda r: r[0] != 'BuildingID') \
              .map(lambda r: (int(r[0]), r[1], int(r[2]), r[3], r[4]))
schema = StructType([StructField('BuildingID', IntegerType(), True),
                     StructField('BuildingMgr', StringType(), True),
                     StructField('BuildingAge', IntegerType(), True),
                     StructField('HVACProduct', StringType(), True),
                     StructField('Country', StringType(), True)])
df = sqlContext.createDataFrame(buildings, schema)

In [10]:
df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACProduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [11]:
# Register the dataframe as a temporary table called HVAC
df.registerTempTable('HVAC')

In [13]:
%%sql
SELECT * FROM HVAC WHERE BuildingAge >= 20

# Convert to RDD

In [13]:
hive_rdd = hivesampletabledf.rdd
hive_rdd.take(3)

[Row(clientid=u'8', querytime=u'18:54:20', market=u'en-US', deviceplatform=u'Android', devicemake=u'Samsung', devicemodel=u'SCH-i500', state=u'California', country=u'United States', querydwelltime=13.9204007, sessionid=0, sessionpagevieworder=0), Row(clientid=u'23', querytime=u'19:19:44', market=u'en-US', deviceplatform=u'Android', devicemake=u'HTC', devicemodel=u'Incredible', state=u'Pennsylvania', country=u'United States', querydwelltime=None, sessionid=0, sessionpagevieworder=0), Row(clientid=u'23', querytime=u'19:19:46', market=u'en-US', deviceplatform=u'Android', devicemake=u'HTC', devicemodel=u'Incredible', state=u'Pennsylvania', country=u'United States', querydwelltime=1.4757422, sessionid=0, sessionpagevieworder=1)]

In [15]:
hive_rdd.filter(lambda x: x.deviceplatform == 'Android').count()

31591

In [16]:
hive_rdd.count()

59793

# Create table in Hive from Data Frame

In [3]:
hivesampletablequerydf = sqlContext.sql("""
SELECT clientid, count(*) as cnt 
FROM hivesampletable 
WHERE state = 'Washington' AND devicemake = 'Microsoft' AND querydwelltime > 15
GROUP BY clientid
""")
hivesampletablequerydf.show()

+--------+---+
|clientid|cnt|
+--------+---+
|  109426|  1|
|    2601|  2|
|   46780|  1|
+--------+---+

In [1]:
%%sql -q
CREATE TABLE IF NOT EXISTS hivesampletablecopypy ( 
                    clientid string, 
                    querytime string, 
                    market string, 
                    deviceplatform string,
                    devicemake string,
                    devicemodel string,
                    state string, 
                    country string,
                    querydwelltime double,
                    sessionid bigint,
                    sessionpagevieworder bigint )

Creating SparkContext as 'sc'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1472684042648_0007,pyspark,idle,Link,Link,✔


Creating HiveContext as 'sqlContext'
SparkContext and HiveContext created. Executing user code ...


In [7]:
from pyspark.sql import DataFrameWriter

dfw = DataFrameWriter(hivesampletabledf)
dfw.insertInto('hivesampletablecopypy', overwrite=True)

In [8]:
%%sql
SHOW TABLES